In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Spring\'23/CSCE670/Animendations/
!ls

/content/drive/MyDrive/Spring'23/CSCE670/Animendations
 animes_synopsis_embeddings  'Project Proposal: Animendations.gdoc'
 data			     'Project Proposal: <Insert Team Name>.gdoc'
 Gojo			      synopsis_autoencoder
 preprocessed_data	     'Untitled spreadsheet.gsheet'


Uncomment the below cell -> Run it -> Comment it -> Restart Runtime -> Run from start

In [ ]:
# !pip install Unidecode
# !pip install word2number
# !pip install contractions
# !pip install transformers
# !python -m spacy download en_core_web_md

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

animes_data_path = "./preprocessed_data/new_preprocessed_animes.csv"
profiles_data_path = "./preprocessed_data/preprocessed_profiles.csv"
reviews_data_path = "./preprocessed_data/new_preprocessed_reviews.csv"

In [ ]:
# reviews_data = pd.read_csv(reviews_data_path)
# reviews_data.head()

In [ ]:
animes_data = pd.read_csv(animes_data_path)

In [ ]:
animes = animes_data.copy()

In [ ]:
animes.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Shounen', 'Sports', 'School', 'Comedy', 'Dra...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Shounen', 'Romance', 'School', 'Music', 'Dra...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Mystery', 'Adventure', 'Fantasy', ...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Shounen', 'Action', 'Military', 'Adventure',...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Mystery', 'Action', 'Vampire', 'Supernatural']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


## Preprocess Synopsis

In [ ]:
animes['synopsis'][0]

"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks.  \r\n \r\nFacing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and its world-class 

In [ ]:
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import unidecode
from word2number import w2n
import contractions
# load spacy model, can be "en_core_web_sm" as well
nlp = spacy.load('en_core_web_md')
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [ ]:
def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

In [ ]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

In [ ]:
def lemmatizeWords(s):
  """Lemmatize words wrt POS tags"""
  s1 = word_tokenize(s)
  # print("After Split:", s1)
  tokens_tag = pos_tag(s1)
  # print("After Token:",tokens_tag)
  word_list = []
  for word, tag in tokens_tag:
    if tag.startswith("NN"):
      word_list.append(lemmatizer.lemmatize(word, pos = 'n'))
    elif tag.startswith("VB"):
      word_list.append(lemmatizer.lemmatize(word, pos = 'v'))
    elif tag.startswith("JJ"):
      word_list.append(lemmatizer.lemmatize(word, pos = 'a'))
    else:
      word_list.append(lemmatizer.lemmatize(word))
  # print(word_list)
  # print(' '.join(word for word in word_list))
  return ' '.join(word for word in word_list)

In [ ]:
def preprocess_text(s):
  """
  1. Strips HTML Tags
  2. remove accented characters from text, e.g. café
  3. expand shortened words, e.g. don't to do not
  4. Lemmatize words wrt POS tags
  5. Generalizes the text by converting words to numbers whenever possible
  6. Removed stopwords except 'no' and 'not'
  7. Removes Symbols - "!@#$%^*_-+=|\;:<>?/.,\'`\"(){}[]"
  Returns: List of cleaned words
  """
  s = str(s).lower()
  s = strip_html_tags(s)
  s = remove_accented_chars(s)
  s = expand_contractions(s)
  s = lemmatizeWords(s)

  try:
    doc = nlp(s)
    # print("doc:", doc)
    tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]
    # print("tokens:", tokens)
    # print(type(tokens[0]))
    s = ' '.join(str(token) for token in tokens)
  except:
    pass

  wordlist = word_tokenize(s)
  
  stop_words = set(stopwords.words('english'))
  stop_words.remove('no')
  stop_words.remove('not')
  filtered_sentence = [w for w in wordlist if not w in stop_words]
  wordlist = filtered_sentence
  # print(wordlist)

  for j in range(len(wordlist)):
      
    symbols = "!@#$%^*_-+=|\;:<>?/.,\'`"

    for i in range(len(symbols)):
      wordlist[j] = wordlist[j].replace(symbols[i], '') ### Removing the symbols

    
    symbols2 = "\"(){}[]"
    for i in range(len(symbols2)):
      wordlist[j] = wordlist[j].replace(symbols2[i], '') ### Removing the symbols

  # print(' '.join(word for word in wordlist if len(word)>=1))
  return ' '.join(word for word in wordlist if len(word)>=1)

In [ ]:
lemmatizer = WordNetLemmatizer()
animes['synopsis'] = animes.apply(lambda x: preprocess_text(x['synopsis']), axis = 1)
animes.head()

<ipython-input-11-82bc1397be00>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,follow participation inter high karasuno high ...,"['Shounen', 'Sports', 'School', 'Comedy', 'Dra...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,music accompany path human metronome prodigiou...,"['Shounen', 'Romance', 'School', 'Music', 'Dra...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,abyss gaping chasm stretch depth earth fill my...,"['Sci-Fi', 'Mystery', 'Adventure', 'Fantasy', ...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,order something obtain something equal value m...,"['Shounen', 'Action', 'Military', 'Adventure',...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,help revive legendary vampire kiss shot acerol...,"['Mystery', 'Action', 'Vampire', 'Supernatural']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


## Synopsis Embedding using "all-distilroberta-v1"

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np

sentence_embeddings_dict = dict()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-distilroberta-v1')
model = AutoModel.from_pretrained('sentence-transformers/all-distilroberta-v1')
model = model.to(device)

def embed_synopsis(uid, sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation = False, return_tensors='pt')
  encoded_input = encoded_input.to(device)
  with torch.no_grad():
      model_output = model(**encoded_input)

  token_embeddings = model_output[0] #First element of model_output contains all token embeddings
  input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
  sentence_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  # sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
  sentence_embeddings = np.array(sentence_embeddings.cpu())
  sentence_embeddings_dict[uid] = sentence_embeddings
  # print(np.array(sentence_embeddings))
  return sentence_embeddings

In [ ]:
animes['synopsis_embedding'] = animes.apply(lambda x: embed_synopsis(x['uid'], [x['synopsis']]), axis = 1)
animes.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,synopsis_embedding
0,28891,Haikyuu!! Second Season,follow participation inter high karasuno high ...,"['Shounen', 'Sports', 'School', 'Comedy', 'Dra...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,"[[0.030100401, 0.020856176, 0.0153466575, 0.00..."
1,23273,Shigatsu wa Kimi no Uso,music accompany path human metronome prodigiou...,"['Shounen', 'Romance', 'School', 'Music', 'Dra...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,"[[0.020662766, -0.03926897, 0.016084887, 0.042..."
2,34599,Made in Abyss,abyss gaping chasm stretch depth earth fill my...,"['Sci-Fi', 'Mystery', 'Adventure', 'Fantasy', ...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,"[[0.020399092, -0.049823172, 0.025280347, 0.00..."
3,5114,Fullmetal Alchemist: Brotherhood,order something obtain something equal value m...,"['Shounen', 'Action', 'Military', 'Adventure',...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,"[[-0.005541601, -0.03639762, 0.022436976, -0.0..."
4,31758,Kizumonogatari III: Reiketsu-hen,help revive legendary vampire kiss shot acerol...,"['Mystery', 'Action', 'Vampire', 'Supernatural']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,"[[0.012214585, -0.04362346, 0.021791369, 0.038..."


In [26]:
# print(sentence_embeddings_dict)

## Training Autoencoder

In [ ]:
animes.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,synopsis_embedding
0,28891,Haikyuu!! Second Season,follow participation inter high karasuno high ...,"['Shounen', 'Sports', 'School', 'Comedy', 'Dra...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,"[[0.030100401, 0.020856176, 0.0153466575, 0.00..."
1,23273,Shigatsu wa Kimi no Uso,music accompany path human metronome prodigiou...,"['Shounen', 'Romance', 'School', 'Music', 'Dra...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,"[[0.020662766, -0.03926897, 0.016084887, 0.042..."
2,34599,Made in Abyss,abyss gaping chasm stretch depth earth fill my...,"['Sci-Fi', 'Mystery', 'Adventure', 'Fantasy', ...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,"[[0.020399092, -0.049823172, 0.025280347, 0.00..."
3,5114,Fullmetal Alchemist: Brotherhood,order something obtain something equal value m...,"['Shounen', 'Action', 'Military', 'Adventure',...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,"[[-0.005541601, -0.03639762, 0.022436976, -0.0..."
4,31758,Kizumonogatari III: Reiketsu-hen,help revive legendary vampire kiss shot acerol...,"['Mystery', 'Action', 'Vampire', 'Supernatural']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,"[[0.012214585, -0.04362346, 0.021791369, 0.038..."


In [23]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

class AE(nn.Module):
    def __init__(self, input_dim=768, encoder_layers = [256, 128, 64], encoding_dim=50, decoder_layers = [64, 128, 256]):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoder_layers[0]),
            nn.ReLU(),
            nn.Linear(encoder_layers[0], encoder_layers[1]),
            nn.ReLU(),
            nn.Linear(encoder_layers[1], encoder_layers[2]),
            nn.ReLU(),
            nn.Linear(encoder_layers[2], encoding_dim),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, decoder_layers[0]),
            nn.ReLU(),
            nn.Linear(decoder_layers[0], decoder_layers[1]),
            nn.ReLU(),
            nn.Linear(decoder_layers[1], decoder_layers[2]),
            nn.ReLU(),
            nn.Linear(decoder_layers[2], input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
      # x = torch.from_numpy(x)
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)
      return encoded, decoded


In [24]:
import gc
gc.enable()
gc.collect()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

model = AE(input_dim=768, encoder_layers = [256, 128, 64], encoding_dim=50, decoder_layers = [64, 128, 256])
model = model.to(device)

loss_function = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, weight_decay = 1e-5, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1, weight_decay = 1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

epochs = 50
# outputs = []
losses = []
epoch_loss = []
all_embeddings = animes['synopsis_embedding'].to_list()
all_embeddings = [torch.from_numpy(embedding) for embedding in all_embeddings]
all_embeddings = [embedding.to(device) for embedding in all_embeddings]

num_embeddings = len(all_embeddings)

for epoch in range(epochs):
  gc.collect()
  loss_per_epoch = 0
  for embedding in all_embeddings:
    reconstructed = model(embedding)[1]
    loss = loss_function(reconstructed, embedding)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_per_epoch += loss
    losses.append(loss)
  epoch_loss = loss_per_epoch/num_embeddings
  scheduler.step(epoch_loss)
  print("------------------------- epoch: {} Loss: {}-------------------------------".format(epoch, epoch_loss))

device: cuda:0
------------------------- epoch: 0 Loss: 0.04200296103954315-------------------------------
------------------------- epoch: 1 Loss: 0.006189187988638878-------------------------------
------------------------- epoch: 2 Loss: 0.029537873342633247-------------------------------
------------------------- epoch: 3 Loss: 0.008786512538790703-------------------------------
------------------------- epoch: 4 Loss: 0.008138451725244522-------------------------------
------------------------- epoch: 5 Loss: 0.013665101490914822-------------------------------
------------------------- epoch: 6 Loss: 0.0012047428172081709-------------------------------
------------------------- epoch: 7 Loss: 0.001186996465548873-------------------------------
------------------------- epoch: 8 Loss: 0.0011857268400490284-------------------------------
------------------------- epoch: 9 Loss: 0.001185173518024385-------------------------------
------------------------- epoch: 10 Loss: 0.0011855058

## Encoding embeddings to 1x50 embeddings

In [51]:
compressed_synopsis_embeddings_dict = dict()

def compressed_embeddings(uid, embedding):
  embedding = torch.from_numpy(embedding)
  embedding = embedding.to(device)
  compressed_embedding = model(embedding)[0]
  compressed_embedding = np.array(compressed_embedding.cpu())
  compressed_synopsis_embeddings_dict[uid] = compressed_embedding

  return compressed_embedding

In [ ]:
all_embeddings = animes['synopsis_embedding'].to_list()
all_embeddings = [torch.from_numpy(embedding) for embedding in all_embeddings]
all_embeddings = [embedding.to(device) for embedding in all_embeddings]

compressed_synopsis_embeddings = [model(embedding)[0] for embedding in all_embeddings]
compressed_synopsis_embeddings_dict = dict()


In [56]:
compressed_synopsis_embeddings_dict = dict()

all_embeddings = sentence_embeddings_dict.values()
all_embeddings = [torch.from_numpy(embedding) for embedding in all_embeddings]
all_embeddings = [embedding.to(device) for embedding in all_embeddings]

compressed_synopsis_embeddings = [model(embedding)[0] for embedding in all_embeddings]

for i, k in enumerate(sentence_embeddings_dict):
    compressed_synopsis_embeddings_dict[k] = compressed_synopsis_embeddings[i]

In [57]:
print(compressed_synopsis_embeddings_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [59]:
import pickle

with open('./synopsis_autoencoder/compressed_synopsis_embeddings.pickle', 'wb') as f1:
    pickle.dump(compressed_synopsis_embeddings_dict, f1)
with open('./synopsis_autoencoder/synopsis_autoencoder_50_epochs_Adam.pickle', 'wb') as f2:
    pickle.dump(model, f2)
with open('./synopsis_autoencoder/sentence_embeddings_dict.pickle', 'wb') as f3:
  pickle.dump(all_embeddings, f3)

In [60]:
model

AE(
  (encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=50, bias=True)
    (7): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=50, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=768, bias=True)
    (7): Sigmoid()
  )
)